
# Import neccessary modules
1 for matrix caculation, 2 for Deep Learning

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from module.beacon_parser import beacon_parser

beacon_list = [f"beacon{idx + 1}" for idx in range(9)]

%matplotlib widget

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import svm
from torch.utils.data import DataLoader, Dataset

# Load Data
python에서 csv파일을 읽어올때, 숫자만 있는것이 아닌 문자열이 포함된 데이터를 읽을 경우 느려지는 것 확인

In [3]:
data_1233_1 = beacon_parser(
    pd.read_csv(
        "./data/1233/note20/customscenario02_20210223173056.csv",
        header=None,
        low_memory=False,
    )
)
data_1233_2 = beacon_parser(
    pd.read_csv(
        "./data/1233/s20/customscenario02_20210223173056.csv",
        header=None,
        low_memory=False,
    )
)
data_1250 = beacon_parser(
    pd.read_csv(
        "./data/1250/customscenario01_20210223150144.csv", header=None, low_memory=False
    )
)
data_1251 = beacon_parser(
    pd.read_csv(
        "./data/1251/customscenario01_20201009224245.csv", header=None, low_memory=False
    )
)
data_list = [data_1233_1, data_1233_2, data_1250, data_1251]

# Gather data
250 row(5초)의 데이터를 모아서, 한 row로 묶어서 처리하였다. RSSI는 60 row중 최댓값을 사용

In [4]:
gathered_list = []
for data in data_list:
    cur_gathered = np.zeros([np.shape(data)[0] // 250, 9])
    for row in range(np.shape(cur_gathered)[0]):
        cur_gathered[row] = np.max(data[row * 250 : row * 250 + 250, -9:], axis=0)
    cur_gathered += 200
    cur_gathered = cur_gathered / 200
    gathered_list.append(cur_gathered)
    print(np.shape(gathered_list[-1]))

(17, 9)
(17, 9)
(358, 9)
(368, 9)


In [5]:
gathered_1233 = np.concatenate((gathered_list[0], gathered_list[1]), axis=0)
gathered_1250 = gathered_list[2]
gathered_1251 = gathered_list[3]
room_data = [gathered_1233, gathered_1250, gathered_1251]
for room in room_data:
    print(np.shape(room))

(34, 9)
(358, 9)
(368, 9)


# "Same Room" Data
같은 방 데이터 끼리 매칭시키고, 그 둘 차의 절댓값으로 구성된 dim = 9의 input  
146 * 146 + 1495 * 1495 + 1535 * 1535 = 4612566

In [6]:
same_1233 = (
    gathered_1233.reshape([1, -1, 9]) - gathered_1233.reshape([-1, 1, 9])
).reshape([-1, 9])
same_1250 = (
    gathered_1250.reshape([1, -1, 9]) - gathered_1250.reshape([-1, 1, 9])
).reshape([-1, 9])
same_1251 = (
    gathered_1251.reshape([1, -1, 9]) - gathered_1251.reshape([-1, 1, 9])
).reshape([-1, 9])
# same_1233 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1233.reshape([-1, 1, 9]), (34, 34, 9)),
#         np.broadcast_to(gathered_1233.reshape([1, -1, 9]), (34, 34, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
# same_1250 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1250.reshape([-1, 1, 9]), (358, 358, 9)),
#         np.broadcast_to(gathered_1250.reshape([1, -1, 9]), (358, 358, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
# same_1251 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1251.reshape([-1, 1, 9]), (368, 368, 9)),
#         np.broadcast_to(gathered_1251.reshape([1, -1, 9]), (368, 368, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
same = np.concatenate((same_1233, same_1250, same_1251))
print(np.shape(same))

(264744, 9)


# "Different Room" Data
다른 방 데이터 끼리 매칭시키고, 그 둘 차의 절댓값으로 구성된 dim = 9의 input  
146 * 1495 + 1495 * 1535 + 1535 * 146 = 2737205

In [7]:
diff_1233_1250 = (
    gathered_1233.reshape([1, -1, 9]) - gathered_1250.reshape([-1, 1, 9])
).reshape([-1, 9])
diff_1250_1251 = (
    gathered_1250.reshape([1, -1, 9]) - gathered_1251.reshape([-1, 1, 9])
).reshape([-1, 9])
diff_1251_1233 = (
    gathered_1251.reshape([1, -1, 9]) - gathered_1233.reshape([-1, 1, 9])
).reshape([-1, 9])
# diff_1233_1250 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1233.reshape([-1, 1, 9]), (34, 358, 9)),
#         np.broadcast_to(gathered_1250.reshape([1, -1, 9]), (34, 358, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
# diff_1250_1251 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1250.reshape([-1, 1, 9]), (358, 368, 9)),
#         np.broadcast_to(gathered_1251.reshape([1, -1, 9]), (358, 368, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
# diff_1251_1233 = np.concatenate(
#     (
#         np.broadcast_to(gathered_1251.reshape([-1, 1, 9]), (368, 34, 9)),
#         np.broadcast_to(gathered_1233.reshape([1, -1, 9]), (368, 34, 9)),
#     ),
#     axis=2,
# ).reshape((-1, 18))
diff = np.concatenate((diff_1233_1250, diff_1250_1251, diff_1251_1233))
print(np.shape(diff))

(156428, 9)


# Loading data for nn

In [8]:
x = np.concatenate((same, diff))
y = np.concatenate((np.zeros([264744, 1]) + 1, np.zeros([156428, 1]))).reshape(-1)

clf = svm.SVC(kernel="poly")
clf.fit(np.concatenate((same, diff)), y)

KeyboardInterrupt: 

In [ ]:
same_result = clf.predict(same)
diff_result = clf.predict(diff)

In [ ]:
true_positive = same[same_result == 1]
false_negative = same[same_result == 0]
false_positive = diff[diff_result == 1]
true_negative = diff[diff_result == 0]
recall = len(true_positive) / (len(true_positive) + len(false_negative))
precision = len(true_positive) / (len(true_positive) + len(false_positive))
accuracy = (len(true_positive) + len(true_negative)) / (
    len(true_positive) + len(false_negative) + len(false_positive) + len(true_negative)
)

In [ ]:
print(
    f"true positive : {len(true_positive)}, false negative : {len(false_negative)}, precision : {precision}"
)
print(
    f"false positive : {len(false_positive)}, true negative : {len(true_negative)}, recall : {recall}"
)
print(f"accuracy : {accuracy}")

In [ ]:
print("false positive")
print(false_positive[:10,:9] * 200 - 200)
print(false_positive[:10, 9:] * 200 - 200)

In [ ]:
print("false negative")
print(false_negative[:10,:9] * 200 - 200)
print(false_negative[:10, 9:] * 200 - 200)

비교분석을 위한 수치  
모든 데이터에 true라고 응답해도, 정확도가 62%는 나온다.

# 개선 계획
학습 과정에서 accuracy가 높아지지 않은 것으로, 제대로 학습되지 않는 상태인것으로 확인  
input에 대한 조절로 결과를 얻고자한다
1. RSSI 값 연산 시 재정의:   
현재 해당 비컨에 해당하는 RSSI가 없으면, 0으로 넣어주고 비교하고있다.  
그결과, null데이터와 -90dB의 차이가, null데이터와 -10dB의 차이보다 더 큰 상태이다.  
null값에 대해 -100을 넣어주고 연산하는 방식으로 오류를 줄여야 한다.  
2. Input 범위 재설정:  
1의 적용 이후, 실제 input으로 들어갈 값들은 -100 ~ -1 사이의 값이다.  
input에 100을 더한 후 100으로 나눠주어, input들이 0 ~ 1 사이의 범위에 있도록 normalize 해줄 계획이다
3. Input Data 추가:  
현재는 두 데이터간의 '차의 절댓값' 만을 넣어주고 있어서, 데이터 dimension이 9이하인 상태이다. (적은 편)  
차만을 사용하는 과정에서, 각 데이터의 RF measurement들이 버려지게 된다.  
4. Input의 다원화:  
현재 사용하고 있지 않은 기압값등을 어떻게 활용할지 고려한다.